In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import StandardScaler 
from kneed import KneeLocator 
from yellowbrick.cluster import silhouette_visualizer 
import scipy.cluster.hierarchy as sch
from sklearn.preprocessing import normalize
from sklearn.cluster import AgglomerativeClustering 

from scipy.stats import f_oneway
import statsmodels.api as sm
from statsmodels.formula.api import ols

from numpy import errstate,isneginf,array
from sklearn.decomposition import PCA
from pandas.plotting import scatter_matrix

import math
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist


Bad key "text.kerning_factor" on line 4 in
C:\Users\tiny2\anaconda3\envs\fastai\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
C:\Users\tiny2\anaconda3\envs\fastai\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


# Importing files

In [2]:
main_df = pd.read_csv('../../Hackerthon_data/DSA3101_Hackathon_Data.csv', parse_dates = ['Date'])
categories = pd.read_csv('../../Hackerthon_data/DSA3101_Hackathon_Categories_Information.csv')
panelists = pd.read_excel('../../Hackerthon_data/DSA3101_Hackathon_Panelists_Demographics.xlsx')

main_df = pd.read_csv('DSA3101_Hackathon_Data.csv', parse_dates = ['Date'])
categories = pd.read_csv('DSA3101_Hackathon_Categories_Information.csv')
panelists = pd.read_excel('DSA3101_Hackathon_Panelists_Demographics.xlsx')

# adding year and month columns
main_df['Year'] = main_df['Date'].dt.year.astype('object')
main_df['Month'] = main_df['Date'].dt.month.astype('object')

df1 = main_df.merge(categories, how = 'left', on='Category')
complete_df = df1.merge(panelists, how = 'left', left_on = 'Panel ID', right_on = 'ID').drop('ID', 1)

# keeping the last 12 months only
df = complete_df[complete_df['Date'] >= '2019-06-14']

# Data Cleaning

In [3]:
# remove rows with spend = 0
df = df[df.Spend != 0]

# change Volume = 0 to spend/price per volume
df['Volume'] = df['Volume'].mask(df["Volume"] == 0, df['Spend']/df["Price per Volume"])

# Feature Engineering

In [4]:
#creating new column, 'InvoiceNo'
df['InvoiceNo'] = df.groupby(['Panel ID', 'Date']).ngroup().astype(str).str.zfill(6)
df['InvoiceNo'] = 'IN_' + df['InvoiceNo']

# assume 1 egg = 50g
# assume 1 instant noodle = 80g
# 1 litre = 1 kg

# creating new column, 'Volume grams' where the unit of measurement is all in grams
df['Volume grams'] = np.where(df['Category']== 'Eggs', 
                                       50.0 * df['Volume'],
                                       1000.0 * df['Volume'])
df['Volume grams'] = np.where(df['Category']== 'Instant Noodles', 
                                       80.0 * df['Volume'],
                                       df['Volume grams'])
df['Volume grams'] = round(df['Volume grams'],2)

# creating new column, 'Total calories'
df['Calories'] = round(df['Calories/100g'] * (df['Volume grams']/100),2)

# creating a is_budget column(if price per volume of category is less than average price ->  is_budget = 1)
df['Is Budget'] = (df['Spend']/(df['Volume']) <= df['Price per Volume'])*1

# Making a Transactions Dataset

In [16]:
transactions = df.groupby('InvoiceNo').agg({'Panel ID': 'first',
                             'Date': 'first',
                             'Category': lambda x: list(x),
                             'Pack Size': 'sum',
                             'Volume grams': 'sum',
                             'Calories': 'sum',
                             'Spend': 'sum',
                             'Year': 'first', 
                             'Month':'first',
                             'BMI':'first',
                             'Income': 'first',
                             'Ethnicity':'first',
                             'Lifestage':'first',
                             'Strata':'first',
                             '#HH':'first',
                             'location':'first'}).reset_index()

# 'Basket Size' is the size of the basket with duplicate items e.g. the person bought 2x Flour
transactions['Basket Size'] = transactions['Category'].apply(len)

# 'Basket' will be a list containing the unique items in the transactions
transactions['Basket'] = transactions['Category'].apply(set).apply(list)

# 'Unique Basket Size' is the size of the UNIQUE basket (no duplicated items)
transactions['Unique Basket Size'] = transactions['Basket'].apply(len)

# drop the 'Category' column
transactions.drop('Category',axis=1, inplace = True)

#each row is an invoice describing that transaction's total calories, total spend, total volume, basket size and unique basket size
transactions = transactions[['Date', 'InvoiceNo', 'Panel ID', 'Calories', 'Spend', 'Volume grams', 'Basket Size', 'Unique Basket Size']] 

## Collaborative filtering
done on the Categories <br>
The frequency of items purchased - the weight <br>
Outputs a list with prediction for each of the panel ID though have not thought of what to do w it yet/ testing with the SVD model

In [66]:
from surprise import Dataset, SVD, accuracy, Reader
from surprise.model_selection import train_test_split


In [75]:
Frequency = df.groupby(['Panel ID','Category'], as_index=False).agg('count')[['Panel ID','Category','Date']].rename(columns = {'Date':'Frequency'})
#Frequency = Frequency.pivot(index = 'Panel ID', columns = 'Category', values = 'Frequency').fillna(0)

In [76]:
Frequency.head()

,Panel ID,Category,Frequency
0,Panel 101011101,Biscuits,8
1,Panel 101011101,Bouilon,3
2,Panel 101011101,Butter,1
3,Panel 101011101,CSD,4
4,Panel 101011101,Canned Product,6


### The SVD model part

The reader value - the min - max frequency purchased(someone bought yogurt 206 times)


In [79]:
reader = Reader(rating_scale=(0,206))
data = Dataset.load_from_df(Frequency[['Panel ID','Category','Frequency']], reader)
trainset, testset = train_test_split(data, test_size=.25)

In [80]:
algo = SVD()
algo.fit(trainset)

In [90]:
predictions = algo.test(testset)

In [91]:
accuracy.rmse(predictions)

RMSE: 7.5864


7.586431552689534

Looks bad though - 

In [98]:
predictions

[Prediction(uid='Panel 138056101', iid='Rice', r_ui=3.0, est=6.4026340838747, details={'was_impossible': False}),
 Prediction(uid='Panel 801548601', iid='CSD', r_ui=4.0, est=2.9845942367223284, details={'was_impossible': False}),
 Prediction(uid='Panel 211096101', iid='Confectionery', r_ui=6.0, est=2.373582216619133, details={'was_impossible': False}),
 Prediction(uid='Panel 242049101', iid='Salad Dressing', r_ui=2.0, est=3.1496778785548, details={'was_impossible': False}),
 Prediction(uid='Panel 801554201', iid='Rice', r_ui=3.0, est=6.651852764288451, details={'was_impossible': False}),
 Prediction(uid='Panel 311055101', iid='Butter', r_ui=1.0, est=2.438653696506502, details={'was_impossible': False}),
 Prediction(uid='Panel 801174801', iid='Ice Cream', r_ui=1.0, est=0.5506481704842043, details={'was_impossible': False}),
 Prediction(uid='Panel 156040101', iid='CSD', r_ui=2.0, est=6.899288528188904, details={'was_impossible': False}),
 Prediction(uid='Panel 801551401', iid='Biscuits',